<a href="https://colab.research.google.com/github/Mojn2000/DeepProject/blob/main/RNN_v2_0(supervised).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

drive_path = 'drive/My Drive/Skole/Uni/9_semester/02456_deep_learning/Project/'

if (not(os.path.exists(drive_path))):
  drive_path = 'drive/MyDrive/DL project/'  # Idas path    

f0 = "padded_data.pickle"

## LOAD PADDED and CLEANED data 
with open(drive_path + f0, 'rb') as f:
    data = pickle.load(f)


     |████████████████████████████████| 256 kB 11.1 MB/s 
Mounted at /content/drive


In [ ]:
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax, leaky_relu, linear 
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Separate into test, train, and bumpy roads 

In [ ]:
# Definition of bumpy road in IRI terms 
tau = 4.5 

good_data = data[data['IRI_mean'] <= tau]
bad_data = data[data['IRI_mean'] > tau]

# Separate good_data into test and train sets 
good_train, good_test = train_test_split(good_data, test_size=0.2)
bad_train, bad_test = train_test_split(bad_data, test_size=0.2)

# Build the model


In [ ]:
train_data = []
test_data = []
test_bad = []

batch_size = 256

# Test data
for i in range(len(good_test['GM.acc.xyz.z'])):
   x_data = good_test['GM.acc.xyz.z'].values[i].flatten() 
   label = 1
   test_data.append([x_data, label])

for i in range(len(bad_test['GM.acc.xyz.z'])):
   x_data = bad_test['GM.acc.xyz.z'].values[i].flatten() 
   label = 0
   test_data.append([x_data, label])

test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

# Train data
for i in range(len(good_train['GM.acc.xyz.z'])):
   x_data = good_train['GM.acc.xyz.z'].values[i].flatten()
   label = 1
   train_data.append([x_data, label])

for i in range(len(bad_train['GM.acc.xyz.z'])):
   x_data = bad_train['GM.acc.xyz.z'].values[i].flatten() 
   label = 0
   train_data.append([x_data, label])

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)


# RNN network

In [ ]:
# Define size variables

num_features = len(good_train['GM.acc.xyz.z'].values[0])
hidden_units = 32 
latent_features = 8 
layer_dim = 2
dropout_prob = 0 


class Supervised(nn.Module):
    def __init__(self, hidden_units, latent_features):
        super(Supervised, self).__init__()

        # ENCODER
        #self.rnn_in = nn.GRU(num_features,(hidden_units*2), layer_dim, batch_first=True, dropout=dropout_prob)
        #self.linear_in1 = nn.Linear(in_features=(hidden_units*2), out_features=hidden_units)
        #self.linear_in2 = nn.Linear(in_features=hidden_units, out_features=latent_features)        
        #self.linear_last = nn.Linear(in_features=latent_features, out_features=1)
        
        self.rnn_in = nn.GRU(num_features,(hidden_units), layer_dim, batch_first=True, dropout=dropout_prob)
        self.linear_in2 = nn.Linear(in_features=hidden_units, out_features=latent_features)        
        self.linear_last = nn.Linear(in_features=latent_features, out_features=1)
        


    def forward(self, x): 
        outputs = {}
        
        # RNN
        x = x[None,:, :]
        x, h = self.rnn_in(x)

        # LINEAR
        x = x[0, :, :]
        #x = relu(self.linear_in1(tanh(x)))
        #x = self.linear_in2(x)

        #x = relu(self.linear_in1(tanh(x)))
        x = self.linear_in2(tanh(x))
        

        x_hat = (tanh(self.linear_last(x))+1)/2
        #x_hat = sigmoid(self.linear_last(x))

        return {
            'x_hat': x_hat
        }


# Choose the shape of the autoencoder
net = Supervised(hidden_units=hidden_units, latent_features=latent_features)

print(net)

Supervised(
  (rnn_in): GRU(3366, 32, num_layers=2, batch_first=True)
  (linear_in2): Linear(in_features=32, out_features=8, bias=True)
  (linear_last): Linear(in_features=8, out_features=1, bias=True)
)


## Set network parameters

In [ ]:
import torch.optim as optim

#OPTIMIZER
optimizer = torch.optim.RMSprop(net.parameters(), lr=1e-3, weight_decay=1e-4)

# Cross entropyloss
def loss_function(yHat, y):
    return ((torch.sum(-torch.log(x_hat[y==1])) + 8*torch.sum(-torch.log(1-x_hat[y==0])))/len(y))

## Train the network 

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

train_loss = []
valid_loss = []

nRuns = 0

In [ ]:
# EPOCHS
num_epochs = 30

# TRAIN MODEL
for epoch in range(num_epochs):
    print("Epoch: ", epoch)
    batch_loss = []
    net.train()
    
    for x,y in train_loader:
        x = x.float()
        outputs = net(x)
        x_hat = outputs['x_hat']

        # LOSS
        loss = loss_function(x_hat, y)
        
        # UPDATE MODEL
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_loss.append(loss.item())

    train_loss.append(np.mean(batch_loss))

    # Evaluate, do not propagate gradients
    with torch.no_grad():
        net.eval()
      
        x, y = next(iter(test_loader))
        x = x.float()
        outputs = net(x)

        x_hat = outputs['x_hat']
       # z = outputs['z'].cpu().numpy()

        loss = loss_function(x_hat, y[:,None].float())

        valid_loss.append(loss.item())
    
    if epoch == 0:
        continue

nRuns += 1

Epoch:  0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29


In [ ]:
# PLOT TRAINING CURVE 
import matplotlib.pyplot as plt
len_epoch = range(len(train_loss))


plt.figure(dpi=800)
plt.plot(len_epoch,np.log10(train_loss),color='r',label="Train loss")
plt.plot(len_epoch,np.log10(valid_loss),color='b', label= "Validation loss")
plt.title("Train Loss")
plt.xlabel("Epochs")
plt.ylabel("MSE error")
plt.grid()
plt.legend()
plt.show()


In [ ]:
torch.mean(x_hat[y==0])

tensor(0.4682)

In [ ]:
prob_good = []
prob_bad = []

for x,y in train_loader:
  x = x.float()
  outputs = net(x)
  x_hat = outputs['x_hat']
  prob_good.extend(x_hat[y==1].flatten().tolist())
  prob_bad.extend(x_hat[y==0].flatten().tolist())

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [ ]:
eps = 1e-6
tau_min = min(min(prob_good),min(prob_bad)) - eps
tau_max = max(max(prob_good),max(prob_bad)) + eps
print(tau_min)
print(tau_max)

N = 100

perf = np.array([np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N)])
ii = 0
for tau in np.linspace(tau_min, tau_max, num = N):
  perf[0,ii] = tau
  perf[1,ii] = sum(prob_good>tau)  # true positives
  perf[2,ii] = sum(prob_good<=tau) # false positive 
  perf[3,ii] = sum(prob_bad>tau)   # false negatives
  perf[4,ii] = sum(prob_bad<=tau) # true negatives
  ii += 1

perf = pd.DataFrame(perf)
perf.to_pickle(drive_path+'RNN_v2_0.pickle')

0.18677921430969238
0.6259317861328125
